# Trabalho Final - Análise de Dados com Python

Integrantes:

1) Vinicius - 345167
2) Guilherme - 346006
3) Fabiana - 345858

### Enunciado

A partir dos dados housing.csv, realize toda análise exploratória necessária e prepare os dados para algoritmos de machine learning usando a Scikit-Learn. Treine um modelo de regressão linear que prediga a variável meta (median_house_value) e cujo valor de RMSE seja menor ou igual a 69000. 

In [307]:
# leitura dos dados
import pandas as pd
housing = pd.read_csv('bases/housing.csv')
housing.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


In [308]:
# linhas e colunas
housing.shape

(20640, 10)

In [309]:
# tipos de dados
housing.dtypes

longitude             float64
latitude              float64
housing_median_age    float64
total_rooms           float64
total_bedrooms        float64
population            float64
households            float64
median_income         float64
median_house_value    float64
ocean_proximity        object
dtype: object

In [310]:
# estatísticas
housing.describe()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
count,20640.000000,20640.000000,20640.000000,20640.000000,20433.000000,20640.000000,20640.000000,20640.000000,20640.000000
mean,-119.569704,35.631861,28.639486,2635.763081,537.870553,1425.476744,499.539680,3.870671,206855.816909
std,2.003532,2.135952,12.585558,2181.615252,421.385070,1132.462122,382.329753,1.899822,115395.615874
min,-124.350000,32.540000,1.000000,2.000000,1.000000,3.000000,1.000000,0.499900,14999.000000
25%,-121.800000,33.930000,18.000000,1447.750000,296.000000,787.000000,280.000000,2.563400,119600.000000
50%,-118.490000,34.260000,29.000000,2127.000000,435.000000,1166.000000,409.000000,3.534800,179700.000000
75%,-118.010000,37.710000,37.000000,3148.000000,647.000000,1725.000000,605.000000,4.743250,264725.000000
max,-114.310000,41.950000,52.000000,39320.000000,6445.000000,35682.000000,6082.000000,15.000100,500001.000000


In [311]:
# nulls
housing.isnull().sum()

longitude               0
latitude                0
housing_median_age      0
total_rooms             0
total_bedrooms        207
population              0
households              0
median_income           0
median_house_value      0
ocean_proximity         0
dtype: int64

In [312]:
# Add total_bedrooms when is null
housing['total_bedrooms'].fillna(housing['total_bedrooms'].mean(), inplace=True)
housing.isnull().sum()

longitude             0
latitude              0
housing_median_age    0
total_rooms           0
total_bedrooms        0
population            0
households            0
median_income         0
median_house_value    0
ocean_proximity       0
dtype: int64

In [313]:
# count value counts for each column
housing['ocean_proximity'].value_counts()


<1H OCEAN     9136
INLAND        6551
NEAR OCEAN    2658
NEAR BAY      2290
ISLAND           5
Name: ocean_proximity, dtype: int64

In [314]:
# normalize ocean_proximity
housing['ocean_proximity'] = housing['ocean_proximity'].map({
    '<1H OCEAN': 1,
    'INLAND': 2,
    'NEAR OCEAN': 3,
    'NEAR BAY': 4,
    'ISLAND': 5
})
housing['ocean_proximity'].value_counts()

1    9136
2    6551
3    2658
4    2290
5       5
Name: ocean_proximity, dtype: int64

In [315]:
# Como só tem 5 valores em ISLAND, vamos remove-los
housing = housing.drop(housing[housing['ocean_proximity'] == 5].index)
housing['ocean_proximity'].value_counts()

1    9136
2    6551
3    2658
4    2290
Name: ocean_proximity, dtype: int64

In [316]:
# Media de pessoas por familia na região
housing['median_population_per_household'] = round(housing['population'] / housing['households'], 2)

In [317]:
# Média de comodos por Familia
housing['median_rooms_per_household'] = round(housing['total_rooms'] / housing['households'], 2)
# Média de quartos por Familia
housing['median_bedrooms_per_household'] = round(housing['total_bedrooms'] / housing['households'], 2)

In [318]:
# Clusterizar por média da idade da casa
housing.loc[housing['housing_median_age'] <= 10, 'housing_age'] = 0
housing.loc[housing['housing_median_age'] > 10, 'housing_age'] = 1
housing.loc[housing['housing_median_age'] > 20, 'housing_age'] = 2
housing.loc[housing['housing_median_age'] > 30, 'housing_age'] = 3
housing.loc[housing['housing_median_age'] > 40, 'housing_age'] = 4
housing.loc[housing['housing_median_age'] > 50, 'housing_age'] = 5


In [324]:
housing.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity,median_population_per_household,median_rooms_per_household,median_bedrooms_per_household,housing_age,group_loc
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,4,2.56,6.98,1.02,4.0,4.0
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,4,2.11,6.24,0.97,2.0,4.0
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,4,2.80,8.29,1.07,5.0,4.0
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,4,2.55,5.82,1.07,5.0,4.0
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,4,2.18,6.28,1.08,5.0,4.0


In [325]:
# Scikit-Learn. Treine um modelo de regressão linear que prediga a variável meta (median_house_value) e cujo valor de RMSE seja menor ou igual a 69000. 
import numpy as np
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
X_train = housing.drop('median_house_value', axis=1)
y_train = housing['median_house_value']
lr.fit(X_train, y_train)
round(lr.score(X_train, y_train) * 100, 2)


63.83

In [326]:
# RSME #69440.13
from sklearn.metrics import mean_squared_error
housing_predictions = lr.predict(X_train)
housing_mse = mean_squared_error(y_train, housing_predictions)
housing_rmse = np.sqrt(housing_mse)
round(housing_rmse, 2)

69380.41